In [49]:
folder = 'data/'
import json
transe = json.load(open(folder + 'embedding.vec.json'))
wv = transe['ent_embeddings.weight']
entities_id = json.load(open(folder + 'ent2ids'))
entities = sorted(entities_id)
print(len(wv), len(entities))
vec = dict()
for i in entities:
    try:
        vec[i] = np.array(wv[entities_id[i]])
    except:
        print(i)
        vec[i] = np.random.rand(100)

29918 29918


In [45]:
test = json.load(open(folder + 'test_tasks.json'))
candidates = json.load(open(folder + 'rel2candidates.json'))

In [50]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import numpy as np
few_shot = 5
neg_sample = 10
np.random.seed(0)
def feature(e1, e2):
#     return np.array(vec[e2]) + np.array(vec[e1])
    return np.concatenate((vec[e1], vec[e2]))
data = dict()
data['AP@50'] = 0
data['AUROC'] = 0
data['AUPRC'] = 0
t = 0
for rel in test:
    X_train = []
    Y_train = []
    Z = []
    seen = set()
    np.random.shuffle(test[rel])
    train_size = min(few_shot, int(len(test[rel]) * 0.8))
    for e1, r, e2 in test[rel][:train_size]:
        X_train.append(feature(e1, e2))
        Y_train.append(1)
        seen.add(e1 + e2)
    neg_pool = []
    for e1 in candidates[rel]:
        for e2 in candidates[rel]:
            if e1 != e2 and (e1 + e2) not in seen and (e2 + e1) not in seen:
                neg_pool.append((e1, e2))
    train_pos = len(X_train)
    np.random.shuffle(neg_pool)
    for e1, e1 in neg_pool[:max(neg_sample, train_pos)]:
        X_train.append(feature(e1, e2))
        Y_train.append(0)
#     print('#train', len(X_train))
    model = LogisticRegression(random_state=0).fit(X_train, Y_train)
    X_test = []
    Y_test = []
    Z = []
    for e1, r, e2 in test[rel][train_size:]: # min(few_shot, int(len(test[rel]) * 0.8))]:
        X_test.append(feature(e1, e2))
        Y_test.append(1)
        seen.add(e1 + e2)
        Z.append((e1, e2))
    pos_num = len(X_test)
    for e1 in candidates[rel]:
        for e2 in candidates[rel]:
            if e1 != e2 and (e1 + e2) not in seen and (e2 + e1) not in seen:
#                 if e1 == 'CID000000271' or e2 == 'CID000000271':
#                     continue
                X_test.append(feature(e1, e2))
                Y_test.append(0)
                Z.append((e1, e2))
    Y_score = model.predict_proba(X_test)[:,1]
    idx = np.argsort(Y_score)[::-1]
    ap50 = 0
    hit = 0
    for i in range(50):
#         print(Z[idx[i]], Y_score[idx[i]])
        if Y_test[idx[i]] == 1:
            hit += 1
            ap50 += hit / (i + 1)
    ap50 /= min(50, pos_num)
    data['AP@50'] += ap50
    fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_score)
    data['AUROC'] += metrics.auc(fpr, tpr)
    precision, recall, thresholds = metrics.precision_recall_curve(Y_test, Y_score)
    data['AUPRC'] += metrics.auc(recall, precision)
    t += 1
    for k in data:
        print(k, data[k] / t, end='   ')
    print()
print('\n Overall')
for k in data:
    print(k, data[k] / t, end='   ')

AP@50 0.0   AUROC 0.6266445592758525   AUPRC 0.015212711793737697   
AP@50 0.0   AUROC 0.606905194966268   AUPRC 0.012548579891086228   
AP@50 0.0   AUROC 0.625627795731959   AUPRC 0.017000815107171163   
AP@50 0.0   AUROC 0.6237498343364196   AUPRC 0.016109564404786697   
AP@50 0.0   AUROC 0.6114540589651869   AUPRC 0.01572774757238824   
AP@50 0.002543791227309933   AUROC 0.6060249313288484   AUPRC 0.024869468823231284   
AP@50 0.005402561910037874   AUROC 0.5966249332487857   AUPRC 0.03447106491815087   
AP@50 0.005222410753408744   AUROC 0.6025966016716726   AUPRC 0.03249825116160991   
AP@50 0.0047115873363633275   AUROC 0.5991722385830364   AUPRC 0.03317066854245602   
AP@50 0.004240428602726995   AUROC 0.595707651629892   AUPRC 0.03095598010101429   
AP@50 0.005602484295482834   AUROC 0.5964728526673415   AUPRC 0.031180329313105503   
AP@50 0.005303960772542766   AUROC 0.5986841784462127   AUPRC 0.029587857641176052   
AP@50 0.004895963790039476   AUROC 0.5985450495548734   AUPR

In [37]:
print(vec['CID000000271'])

[-0.05174468457698822, 0.030864080414175987, 0.030649293214082718, 0.02212749421596527, 0.01835193671286106, -0.008650850504636765, -0.03983186185359955, -0.0554875023663044, 0.05800887569785118, -0.05075596272945404, 0.04195448383688927, 0.0004295983526390046, 0.004965443164110184, -0.03526724502444267, 0.051659148186445236, 0.04479290917515755, 0.0696185752749443, 0.0395987443625927, -0.03921341150999069, -0.015454343520104885, -0.011899737641215324, 0.04733050987124443, 0.04937082156538963, 0.0204714797437191, 0.04255235195159912, 0.013186671771109104, 0.05647919327020645, -0.05039578676223755, 0.061034299433231354, -0.07714084535837173, -0.021564088761806488, -0.015633851289749146, -0.017235377803444862, -0.06416943669319153, 0.020534148439764977, 0.02405054122209549, -0.08051585406064987, -0.04604629799723625, 0.0051735215820372105, -0.029846059158444405, -0.0015281358500942588, 0.07666517049074173, 0.047486502677202225, 0.03327467292547226, -0.01684613898396492, -0.04970105737447